# Probabilistic Graphical Models - Homework 2, Gaussian Process

MVA 2021-2022

Elías Masquil (eliasmasquil@gmail.com)

Nicolás Violante (nviolante96@gmail.com)


# Gaussian Process

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/probabilistic_graphical_models/UScrimes.csv'
TRAINING_FRACTION = 0.75

data = pd.read_csv(DATA_PATH)
last_training_idx = int(len(data) * TRAINING_FRACTION)

X = np.array([data["Assault"].to_numpy(), data["UrbanPop"].to_numpy()]).T
y = data["Murder"].to_numpy()
X_train = X[:last_training_idx]
X_test = X[last_training_idx:]
y_train = y[:last_training_idx]
y_test = y[last_training_idx:]

y_mean = y_train.mean()
y_std = y_train.std()
y_train = (y_train - y_mean) / y_std 
y_test = (y_test - y_mean) / y_std
X_mean = X_train.mean()
X_std = X_train.std()
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std 

## Implementation

We implement both the learning and prediction functions in a standalone GuassianProcess class

In [ ]:
class GuassianProcess:
    def __init__(self):
        self.params = np.ones(5)

    def fit(self, X, y, num_iters, lr):
        N = X.shape[0]
        y = y[:, None]
        L_list = np.zeros(num_iters)
        for t in range(num_iters):
            # Compute Kernel and C
            term = X@X.T
            sum = (X**2).sum(1)[:, None]
            K = self.params[0] * np.exp(- 0.5 * self.params[1] * (sum + sum.T - 2*term))
            K += self.params[2] + self.params[3] * term
            C = K + (np.eye(N) * self.params[4])
            C_inv = np.linalg.inv(C) 

            # Likelihood
            L = - 0.5 * np.log(np.linalg.det(C)) - 0.5 * y.T@C_inv@y - 0.5 * N * np.log(2*np.pi)
            L_list[t] = L

            if t % 100 == 0:
                print(f"[{t}/{num_iters}] likelihood {L_list[t]}")
                print(f"params {self.params}")
            
            # Derivatives
            derivatives = np.zeros((5, N, N))
            derivatives[0] = np.ones((N, N))
            derivatives[1] = term
            derivatives[2] = np.exp(- 0.5 * self.params[1] * (sum + sum.T - 2*term))
            for i in range(N):
                for j in range(N):
                    derivatives[3, i, j] = - self.params[0] * np.exp(-0.5 * self.params[1] *np.linalg.norm(X[i] - X[j], ord=2)**2) * 0.5 * np.linalg.norm(X[i] - X[j], ord=2)**2
            derivatives[4] = np.eye(N)

            # Gradient descent updates
            for i, d in enumerate(derivatives):
                grad =  - 0.5 * np.trace(C_inv @ d) + 0.5 * y.T @ C_inv @ d @ C_inv @ y
                self.params[i] =  self.params[i] + lr * grad

            self.C_inv = C_inv
            self.X_train = X
            self.y_train = y

    def predict(self, X):
        N = len(self.y_train)
        y_pred = np.zeros(X.shape[0])
        for n, x in enumerate(X):
            k = np.zeros(N)
            for i in range(N):
                k[i] = self.params[0] * np.exp(-0.5 * self.params[1] * np.linalg.norm(self.X_train[i] - x, ord=2)**2)
                k[i] += self.params[2] + self.params[3] * np.dot(x, self.X_train[i]) 
            
            y_pred[n] = k[:, None].T @ self.C_inv @ self.y_train
            

        return y_pred

In [ ]:
def rmse(y_pred, y_true):
    return np.sqrt(np.mean((y_pred - y_true)**2))

## Real data
We compare the performance, in terms on RMSE, of our Gaussian Process model and a Linear Regression. We observe a slighly better performance of the Gaussian Process model in both the training and the test set.

In [ ]:
# Train Gaussian Process model
gp = GuassianProcess()
gp.fit(X_train, y_train, num_iters=500, lr=0.001)

[0/500] likelihood -46.46850255117383
params [1. 1. 1. 1. 1.]
[100/500] likelihood -39.97144204195021
params [0.97891901 0.94673145 0.90621878 0.90999745 0.34941264]
[200/500] likelihood -39.83948189459083
params [0.95662199 0.88925312 0.82010819 0.82864426 0.35026772]
[300/500] likelihood -39.699665680080464
params [0.9332891  0.82851876 0.73737385 0.7453523  0.35143642]
[400/500] likelihood -39.553092147816656
params [0.90883166 0.76422419 0.65883625 0.66125807 0.35265967]


In [ ]:
y_train_pred = gp.predict(X_train)
y_test_pred = gp.predict(X_test)

In [ ]:
# Train Linear Regression for comparison
linear = LinearRegression()
linear.fit(X_train, y_train)
y_train_linear = linear.predict(X_train)
y_test_linear = linear.predict(X_test)

print(f"RMSE: Gaussian Process, train {rmse(y_train_pred, y_train):.3f}, test: {rmse(y_test_pred, y_test):.3f}")
print(f"RMSE: Linear Regression, train {rmse(y_train_linear, y_train):.3f} test: {rmse(y_test_linear, y_test):.3f}")

RMSE: Gaussian Process, train 0.555, test: 0.525
RMSE: Linear Regression, train 0.593 test: 0.547
